In [3]:
from sklearn.metrics import f1_score
import mlflow
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
import numpy as np

In [4]:
X_test = torch.load("X_test.pt")
X_train = torch.load("X_train.pt")
y_test = torch.load("y_test.pt")
y_train = torch.load("y_train.pt")

In [5]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.mps.is_available():
    device = 'mps'
device

'mps'

In [6]:
class MyModel(nn.Module):
    def __init__(
        self,
        dataloader: DataLoader,
        input_dim: int,
        output_dim: int = 1,
        dropout_rate: float = 0.5,
    ):
        super(MyModel, self).__init__()
        self.dataloader = dataloader
        self.conv1d = nn.Conv1d(in_channels=input_dim, out_channels=384, kernel_size=3, stride=1, padding=2)
        self.maxpooling1d = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.last_linear = nn.Linear(384, output_dim)
        self.sigmoid = nn.Sigmoid()

        self.to(device)

    def forward(self, x: torch.Tensor):
        if len(x.shape) == 2:  # Если x имеет размерность [batch_size, sequence_length]
            x = x.unsqueeze(1)  # Добавляем измерение каналов: [batch_size, 1, sequence_length]
        x = x.permute(0, 2, 1)  # [batch_size, input_dim, sequence_length]
        x = self.conv1d(x)
        x = self.maxpooling1d(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = x.permute(0, 2, 1)  # Возвращаем размерность [batch_size, sequence_length, features]
        x = self.last_linear(x)
        x = self.sigmoid(x)
        return x

    def fit(
        self,
        num_epoch: int = 10,
        lr: float = 1e-10,
    ):
        self.epoch_loss = []
        self.batch_loss = []
        self.f1_score = []
        self.all_labels = []
        self.all_probs = []
        criterion = nn.BCELoss()  # Бинарная кросс-энтропия
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(num_epoch):
            self.train()
            epoch_losses = []
            for batch_X, batch_y in tqdm(self.dataloader, desc=f"Epoch [{epoch+1}/{num_epoch}]; learning state\t"):
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                optimizer.zero_grad()
                outputs = self(batch_X)
                loss: torch.Tensor = criterion(outputs.squeeze(1), batch_y.unsqueeze(1))
                loss.backward()
                optimizer.step()
                epoch_losses.append(loss.item())
                self.batch_loss.append(loss.item())
            
            self.epoch_loss.append(sum(epoch_losses) / len(epoch_losses))
            
            # Вычисление метрик
            self.eval()
            with torch.no_grad():
                self.all_preds = []
                self.all_labels = []
                for batch_X, batch_y in tqdm(self.dataloader, desc=f"Epoch [{epoch+1}/{num_epoch}]; metric state\t"):
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    outputs = self(batch_X).squeeze(1)
                    preds = (outputs > 0.5).float()
                    self.all_preds.extend(preds.squeeze(1).cpu().numpy())
                    self.all_labels.extend(batch_y.cpu().numpy())

                self.all_labels = np.array(self.all_labels, dtype=int)
                self.all_preds = np.array(self.all_preds, dtype=int)
                f1 = f1_score(self.all_labels, self.all_preds)
                self.f1_score.append(f1)
                print(f"Epoch [{epoch+1}/{num_epoch}], Loss: {loss.item():.4f}, F1 score: {f1:.5f}")
    
    def plot_metrics(self, X_test: torch.Tensor, y_test:torch.Tensor):
        # plt.figure(figsize=(16, 5))
        # plt.subplot(1, 3, 1)
        # plt.plot(self.epoch_loss, label="Loss")
        # plt.xlabel("Epoch")
        # plt.ylabel("Loss")
        # plt.title("Loss per Epoch")
        # plt.legend()

        # plt.subplot(1, 3, 2)
        # plt.plot(self.batch_loss, label="Loss")
        # plt.xlabel("Batches")
        # plt.ylabel("Loss")
        # plt.title("Loss per batch")
        # plt.legend()

        # # График F1-score
        # plt.subplot(1, 3, 3)
        # plt.plot(self.f1_score, label="F1-score", color="orange")
        # plt.xlabel("Epoch")
        # plt.ylabel("F1-score")
        # plt.title("F1-score per Epoch")
        # plt.legend()

        X_test = X_test.float()
        X_test = X_test.unsqueeze(1)
        X_test = X_test.to(device)
        self.eval()
        with torch.no_grad():
            preds = self(X_test)
        preds_squeezed = preds.squeeze(1).squeeze(1)
        result_pres = (preds_squeezed > 0.5).float()
        return f1_score(y_test.numpy(), result_pres.cpu().numpy())
        # print(f"F1 score on test tensors is: {f1*100:.2f}")

In [7]:
# не изменные константы для модельки
input_dim = X_train.shape[1]
output_dim = 1
dataset = TensorDataset(X_train, y_train.float())

In [8]:
dropout_rate = 0.2
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
model = MyModel(dataloader, input_dim, output_dim, dropout_rate)

dropout_rate = 0.6
dataloader_1 = DataLoader(dataset, batch_size=128, shuffle=True)
model_1 = MyModel(dataloader_1, input_dim, output_dim, dropout_rate)

dropout_rate = 0.3
dataloader_2 = DataLoader(dataset, batch_size=512, shuffle=True)
model_2 = MyModel(dataloader_2, input_dim, output_dim, dropout_rate)

dropout_rate = 0.2
dataloader_3 = DataLoader(dataset, batch_size=1024, shuffle=True)
model_3 = MyModel(dataloader_3, input_dim, output_dim, dropout_rate)

In [10]:
models = {
    "DR02_BS128_EP150_LRE-7":  [model,   150, 1e-7],
    "DR06_BS128_EP50_LRE-3":  [model_1,  50, 1e-3],
    "DR03_BS512_EP150_LRE-3": [model_2, 150, 1e-3],
    "DR02_BS1024_EP50_LRE-3": [model_3,  50, 1e-3],
}

mlflow.set_experiment("Toxic_Comments_Model_Comparison")

for model_name, model_args in models.items():
    with mlflow.start_run(run_name=model_name):
        fitting_model: MyModel = model_args[0]
        fitting_model.fit(lr=model_args[2], num_epoch=model_args[1])
        test_f1 = fitting_model.plot_metrics(X_test, y_test)
        mlflow.log_param("model", model_name)
        mlflow.log_metrics({
            "f1s": fitting_model.f1_score[-1],
            "epochs_errors": fitting_model.epoch_loss[-1],
            "batch_errors": fitting_model.batch_loss[-1],
            "test_f1": test_f1,
        })
        mlflow.pytorch.log_model(model, f"model_{model_name}")

Epoch [1/150]; learning state	:   0%|          | 0/1527 [00:00<?, ?it/s]

Epoch [1/150]; metric state	:   0%|          | 0/1527 [00:00<?, ?it/s]

Epoch [1/150], Loss: 0.1789, F1 score: 0.63045


Epoch [2/150]; learning state	:   0%|          | 0/1527 [00:00<?, ?it/s]

Epoch [2/150]; metric state	:   0%|          | 0/1527 [00:00<?, ?it/s]

...

Epoch [49/50], Loss: 0.0474, F1 score: 0.98664


Epoch [50/50]; learning state	:   0%|          | 0/191 [00:00<?, ?it/s]

Epoch [50/50]; metric state	:   0%|          | 0/191 [00:00<?, ?it/s]

Epoch [50/50], Loss: 0.0402, F1 score: 0.99048


2025/05/06 13:03:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [12]:
!mlflow ui

[2025-05-06 14:04:20 +0300] [91602] [INFO] Starting gunicorn 23.0.0
[2025-05-06 14:04:20 +0300] [91602] [INFO] Listening at: http://127.0.0.1:5000 (91602)
[2025-05-06 14:04:20 +0300] [91602] [INFO] Using worker: sync
[2025-05-06 14:04:20 +0300] [91603] [INFO] Booting worker with pid: 91603
[2025-05-06 14:04:20 +0300] [91604] [INFO] Booting worker with pid: 91604
[2025-05-06 14:04:20 +0300] [91605] [INFO] Booting worker with pid: 91605
[2025-05-06 14:04:20 +0300] [91606] [INFO] Booting worker with pid: 91606
^C

Aborted!
[2025-05-06 14:05:21 +0300] [91604] [INFO] Worker exiting (pid: 91604)
[2025-05-06 14:05:21 +0300] [91605] [INFO] Worker exiting (pid: 91605)
[2025-05-06 14:05:21 +0300] [91603] [INFO] Worker exiting (pid: 91603)
[2025-05-06 14:05:21 +0300] [91606] [INFO] Worker exiting (pid: 91606)
